In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
from sentence_transformers import SentenceTransformer
import spacy
from tqdm import tqdm
from collections import defaultdict
from datasets import load_metric, load_dataset, load_from_disk
from differ import diff_ratio
import re
import seaborn as sns
import json
from tqdm import tqdm

In [3]:
os.environ['HF_DATASETS_CACHE'] = '/mnt/cerberus-gpu-data/data/users/k1897038/mimic_summarisation/hf_cache_dir/datasets'
os.environ['TRANSFORMERS_CACHE'] = '/mnt/cerberus-gpu-data/data/users/k1897038/mimic_summarisation/hf_cache_dir/transformers'

In [97]:
# ds_path = '../../mimic_summ_data/mimic_3_val_filt.json'
ds_path = '../../mimic_summ_data/mimic_3_test.json'

In [5]:
ds_path = '../../mimic_summ_data/mimic_3_train.json'

In [6]:
ds = load_dataset('json', data_files=ds_path)['train']

Using custom data configuration default-79c5427ab5e0eba1


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/json/default-79c5427ab5e0eba1/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
pat = re.compile(r'(?:brief)?\n?\s?hospital course:?|SUMMARY OF HOSPITAL COURSE BY SYSTEMS:?', re.IGNORECASE)
def clean(s: str):
    s = s.replace('\n\n' ,'\n')\
         .replace(r'\s{2+}', ' ')\
         .replace(r'\t', ' ')
    return pat.sub('', s)

In [8]:
ds = ds.map(lambda d: {'clean_sum': clean(d['summary'])}, num_proc=5)

#1:   0%|          | 0/7671 [00:00<?, ?ex/s]

#2:   0%|          | 0/7671 [00:00<?, ?ex/s]

#0:   0%|          | 0/7672 [00:00<?, ?ex/s]

#4:   0%|          | 0/7671 [00:00<?, ?ex/s]

#3:   0%|          | 0/7671 [00:00<?, ?ex/s]

In [ ]:
ds = ds.map(lambda d: {'clean_text': clean(d['text'])}, num_proc=10)

#0:   0%|          | 0/3836 [00:00<?, ?ex/s]

#1:   0%|          | 0/3836 [00:00<?, ?ex/s]

#2:   0%|          | 0/3836 [00:00<?, ?ex/s]

#3:   0%|          | 0/3836 [00:00<?, ?ex/s]

#6:   0%|          | 0/3835 [00:00<?, ?ex/s]

#4:   0%|          | 0/3836 [00:00<?, ?ex/s]

#5:   0%|          | 0/3836 [00:00<?, ?ex/s]

#8:   0%|          | 0/3835 [00:00<?, ?ex/s]

#7:   0%|          | 0/3835 [00:00<?, ?ex/s]

#9:   0%|          | 0/3835 [00:00<?, ?ex/s]

In [ ]:
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('sentencizer')

In [ ]:
tok = nlp.tokenizer
sentensizer = nlp.pipeline[-1][-1]
def find_sents(text):
    return [s.text for s in sentensizer(tok(text)).sents if len(s) > 3]

In [ ]:
ds = ds.map(lambda d: {'summ_sents': find_sents(d['clean_sum'])}, num_proc=10)

In [ ]:
ds = ds.map(lambda d: {'text_sents': find_sents(d['clean_text'])},  num_proc=10)

In [ ]:
# train 6 seperate LSTM models for prediciting 'top-line' extractive summaries.
sent_limits = [1,2,3,5,10,15]

In [74]:
ds

Dataset({
    features: ['hadm_id', 'summary', 'text', 'clean_sum', 'clean_text', 'summ_sents', 'text_sents', 'summ_lim_1'],
    num_rows: 38356
})

In [ ]:
sent_limd_sums = defaultdict(list)
for lim in tqdm(sent_limits):
    ds = ds.map(lambda d: {f'summ_lim_{lim}': d['summ_sents'][:lim]}, num_proc=10)

In [ ]:
def _parse_score(lvl, scores):
    return (lvl, scores[lvl].mid.precision, scores[lvl].mid.recall, scores[lvl].mid.fmeasure)

In [ ]:
input_lens = ds.map(lambda d: {'lens': len(d['text_sents'])}, num_proc=10)

In [ ]:
sns.histplot(input_lens['lens'], stat='count', bins=50)

In [ ]:
len([i for i in input_lens['lens'] if i < 1000]) / len(ds)

In [ ]:
# take the top 500 of start, end of sentences. For 93% of cases this is the full amount...

In [28]:
def top_tail_texts(d, col_name):
    sents = d[col_name]
    if len(sents) > 1000:
        first = d[col_name][:500]
        last = d[col_name][-500:]
        sents = first + last
    return sents

In [29]:
ds = ds.map(lambda d: {'text_sents_limd': top_tail_texts(d, 'text_sents')}, num_proc=10)

#0:   0%|          | 0/3836 [00:00<?, ?ex/s]

#1:   0%|          | 0/3836 [00:00<?, ?ex/s]

#2:   0%|          | 0/3836 [00:00<?, ?ex/s]

#3:   0%|          | 0/3836 [00:00<?, ?ex/s]

#6:   0%|          | 0/3835 [00:00<?, ?ex/s]

#5:   0%|          | 0/3836 [00:00<?, ?ex/s]

#4:   0%|          | 0/3836 [00:00<?, ?ex/s]

#9:   0%|          | 0/3835 [00:00<?, ?ex/s]

#7:   0%|          | 0/3835 [00:00<?, ?ex/s]

#8:   0%|          | 0/3835 [00:00<?, ?ex/s]

In [ ]:
first_sents = ds['text_sents_limd'][1]

In [30]:
ds

Dataset({
    features: ['hadm_id', 'summary', 'text', 'clean_sum', 'clean_text', 'summ_sents', 'text_sents', 'summ_lim_1', 'summ_lim_2', 'summ_lim_3', 'summ_lim_5', 'summ_lim_10', 'summ_lim_15', 'text_sents_limd'],
    num_rows: 38356
})

In [31]:
ds.cleanup_cache_files()

10

In [32]:
ds.save_to_disk('mimic3_train_dataset_pre_processed')

In [36]:
ds

Dataset({
    features: ['hadm_id', 'summary', 'text', 'clean_sum', 'clean_text', 'summ_sents', 'text_sents', 'summ_lim_1', 'summ_lim_2', 'summ_lim_3', 'summ_lim_5', 'summ_lim_10', 'summ_lim_15', 'text_sents_limd'],
    num_rows: 38356
})

In [ ]:
# only encode the limited text_sents.
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def encode(sents):
    return [model.encode(s) for s in sents]
ds = ds.map(lambda d:{'text_embed_limd': encode(d['text_sents_limd'])})

In [18]:
ds = load_from_disk('../../mimic_summ_data/mimic3_test_dataset_pre_processed_with_sent_encoding')

In [ ]:
ds = ds.map(lambda d: {'text_embed_len': len(d['text_sents_limd'])})

In [20]:
ds['text_embed_len'][0:5]

[369, 493, 411, 135, 377]

In [ ]:
[len(d) for d in ds['text_embed_limd'][0:5]]

In [37]:
ds = load_from_disk('mimic3_val_dataset_pre_processed_with_sent_encoding')

In [52]:
ds.save_to_disk('mimic3_train_dataset_pre_processed')

In [ ]:
for lim in tqdm(sent_limits):
    col = f'summ_lim_{lim}'
    def map_preds(d):
        summ_sents = d[col]
        text_sents = d['text_sents_limd']
        sim_text_sent_idxs = []
        sents_to_compare = text_sents
        for summ_sent in summ_sents:
            sents_to_compare = [s if i not in sim_text_sent_idxs else '' 
                                for i, s in enumerate(text_sents)]
            ratios = [diff_ratio(summ_sent, sent)[0] for sent in sents_to_compare]
            max_ratio_sents = np.where(ratios == np.amax(ratios))[0]
            sim_text_sent_idxs.extend(max_ratio_sents)
        preds = np.zeros(len(text_sents))
        for i in sim_text_sent_idxs:
            preds[i] = 1
        return preds
    ds = ds.map(lambda d: {f'preds_lim_{lim}': map_preds(d)}, num_proc=10) 

In [136]:
nlp = spacy.load('en_core_web_md')

In [ ]:
ds = ds.map(lambda d: {'text_embed_limd_spacy': [nlp(s).vector for s in d['text_sents_limd']]})

In [10]:
ds.save_to_disk('/data/users/k1897038/mimic3_dataset_pre_processed')

In [7]:
metric = load_metric('rouge')

In [10]:
sent_limits = [1,2,3,5,10,15]

In [26]:
extractive_score_ceil = {}
for lim in tqdm(sent_limits):
    text_sums = []
    for sent_idxs, text_sents in zip([np.argwhere(np.array(p) == 1) for p in ds[f'preds_lim_{lim}']], 
                                     ds['text_sents']):
        text_sum = []
        for i in sent_idxs:
            t = text_sents[i[0]]
            if t not in text_sum:
                text_sum.append(t)
        text_sums.append(''.join(text_sum))
    metric.add_batch(predictions=text_sums, references=[''.join(s) for s in ds[f'summ_lim_{lim}']])
    scores = metric.compute()
    extractive_score_ceil[lim] = _parse_score('rouge1', scores), _parse_score('rouge2', scores), _parse_score('rougeLsum', scores)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [2:40:54<00:00, 1609.12s/it]


In [27]:
extractive_score_ceil

{1: (('rouge1', 0.2954789037242724, 0.3105789838775548, 0.26893790052938443),
  ('rouge2', 0.1530013865553545, 0.18564336298053785, 0.14916766696966413),
  ('rougeLsum', 0.2869088898344726, 0.30200243852475417, 0.2611722859785751)),
 2: (('rouge1', 0.32752561931874313, 0.32203975278645547, 0.29883012290461897),
  ('rouge2', 0.15113092568450023, 0.16597535441556202, 0.14596105283919245),
  ('rougeLsum', 0.3149819334644566, 0.3105165310723039, 0.28765856215926366)),
 3: (('rouge1', 0.3376198158858904, 0.32930108144291087, 0.31184174913281104),
  ('rouge2', 0.1449426198825109, 0.15423277917724654, 0.14023967984408575),
  ('rougeLsum', 0.32468482348078204, 0.3175217381467632, 0.30019911003394284)),
 5: (('rouge1', 0.3468834457085018, 0.3392393929861849, 0.32352373382159033),
  ('rouge2', 0.13473345442479184, 0.1401618027871774, 0.1302094035482527),
  ('rougeLsum', 0.33473549433230215, 0.3280669965165318, 0.31245517642187637)),
 10: (('rouge1', 0.3529488550942286, 0.35237821726160845, 0.331

In [38]:
json.dump(extractive_score_ceil, open('/data/users/k1897038/outputs/extractive_score_ceil.json', 'w'))